 <img src="https://datacube.remote-sensing.org/wp-content/uploads/2021/03/cube_namibia.png" align="left" width='70' alt="sa_logo">
          <img src="https://datacube.remote-sensing.org/wp-content/uploads/2021/02/ci_logo_navbar.png" align="right" width='180' alt="eo_logo">

# NSA Data Cube Workshop

In [ ]:
import datacube

import geopandas as gpd
from odc.ui import with_ui_cbk
import xarray as xr
import seaborn as sns
from eo2cube_tools import plot
import pandas as pd
from dea_tools.plotting  import display_map, rgb
from eo2cube_tools.spectralindices import spectralindices

## Connect to datacube

In [ ]:
dc = datacube.Datacube(app = 'nsa')
dc

### Browse the available Data Cubes

In [ ]:
list_of_products = dc.list_products()

netCDF_products = list_of_products[list_of_products['format'] == 'NetCDF']

netCDF_products

### Datacube Measurements

In [ ]:
dc.list_measurements()

# Working with Sentinel-2

### Inspecting data

In [ ]:
gdf = gpd.read_file('example_data/walvis_bay.shp')
plot.map_polygon(gdf)

In [ ]:
longitude_extents = (gdf.total_bounds[0] , gdf.total_bounds[2] )
latitude_extents  = (gdf.total_bounds[1] , gdf.total_bounds[3] )

In [ ]:
# select time period
time_extents=('2020-02-01', '2020-02-06')

# select platform
platform = "SENTINEL_2"

# select product
product = "s2_l2a_namibia"

dataset = dc.load(latitude = latitude_extents,
                             longitude = longitude_extents,
                             time = time_extents,
                             platform =platform,
                             skip_broken_datasets=True,
                             measurements = ['red','green','blue', 'nir', 'scl'],
                             product = product,
                             group_by = "solar_day",
                             progress_cbk=with_ui_cbk()) 

In [ ]:
rgb(dataset.isel(time=[0]), bands=['red', 'green', 'blue'], col="time")

### Manipulating data

In [ ]:
gdf = gpd.read_file('example_data/windhoek.shp')
plot.map_polygon(gdf)

In [ ]:
longitude_extents = (gdf.total_bounds[0] , gdf.total_bounds[2] )
latitude_extents  = (gdf.total_bounds[1] , gdf.total_bounds[3] )

# select time period
time_extents=('2020-02-01', '2020-04-29')

# select platform
platform = "SENTINEL_2"

# select product
product = "s2_l2a_namibia"

dataset = dc.load(latitude = latitude_extents,
                             longitude = longitude_extents,
                             time = time_extents,
                             platform =platform,
                             skip_broken_datasets=True,
                             measurements = ['red','green','blue', 'nir', 'scl','swir1','swir2'],
                             product = product,
                             group_by = "solar_day",
                             progress_cbk=with_ui_cbk()) 

In [ ]:
mean = dataset.mean(dim=['x','y'])
dataset = dataset.sel(time = pd.to_datetime(mean.time.where(mean['red'] !=0, drop=True).values.tolist()))

## Inspect dataset

In [ ]:
dataset

In [ ]:
dataset.nir

In [ ]:
dataset.isel(time =7)

In [ ]:
dataset.isel(time=1).red.mean()

In [ ]:
dataset.isel(time=[0,1,2])

In [ ]:
dataset.sel(time=slice("2020-02-08","2020-02-25"))

## Plotting

In [ ]:
dataset.nir.isel(time=7).plot(figsize=(12,9), cmap='Greys')

In [ ]:
dataset.red.isel(time=[1,3,5]).plot(col="time", figsize=(17,5))

In [ ]:
dataset.red.isel(time=[2,3]).plot(col="time", robust = True ,cmap ='magma', col_wrap = 2, figsize = (10,5))

In [ ]:
from eo2cube_tools.plot import plot_band

plot_band(dataset)

In [ ]:
rgb(dataset.isel(time=[2]), bands=['red', 'green', 'blue'], col="time")

In [ ]:
rgb(dataset.isel(time=[2]), bands=['swir1', 'blue', 'green'], col="time")

In [ ]:
from eo2cube_tools.plot import plot_rgb

plot_rgb(dataset)

In [ ]:
from dea_tools.plotting import xr_animation

# Produce time series animation of red, green and blue bands
xr_animation(ds=dataset, 
             bands=['red', 'green', 'blue'],
             output_path='animated_timeseries.gif',                                
             interval=800, 
             width_pixels=300)

# Plot animated gif
plt.close()

In [ ]:
from IPython.display import Image

Image(filename='animated_timeseries.gif')

## Simple band math

In [ ]:
#Calculate NDVI
ndvi = (dataset.nir - dataset.red)/(dataset.nir + dataset.red)
ndvi

In [ ]:
dataset['NDVI'] = ndvi
dataset

In [ ]:
dataset.NDVI.isel(time= 6).plot(cmap = 'Greens', figsize = (10,7))

In [ ]:
dataset = spectralindices(dataset, indices=['EVI','NDWI', 'LAI','NBR'])

In [ ]:
dataset.NBR.isel(time= 2).plot(cmap = 'Blues', figsize = (10,7))

In [ ]:
mean = dataset.NDVI.mean(dim=['x','y'])
mean.plot()

## Cloud removal using the SCL-Band

In [ ]:
rgb(dataset, bands=['red', 'green', 'blue'], index = [0,4], percentile_stretch = [0.05, 0.95])

In [ ]:
dataset.scl.flags_definition

In [ ]:
dataset.scl.isel(time=[0,4]).plot(col="time", figsize=(10,8))

In [ ]:
def mask(dataset): #input options: low - filter out cloud of high probability only (default); high - filter out cloud of high and medium probability
    cloud_free = dataset.where((dataset.scl != 9) & (dataset.scl != 3))
    new_ds = cloud_free.where((dataset.blue < 12000) & (dataset.red < 12000) & (dataset.green < 12000)) #pixels of good quality
    return new_ds

ds_nc = mask(dataset)
ds_nc

In [ ]:
rgb(ds_nc, bands=['red', 'green', 'blue'], index = [0,4])

## Creating composites

In [ ]:
timesteps = [0, 1, 2, 3, 4, 5]
rgb(dataset,bands=['red', 'green', 'blue'], index=timesteps)

In [ ]:
ds_median = ds_nc.median('time')
rgb(ds_median, bands=['red', 'green', 'blue'])

In [ ]:
ds_resampled_median = ds_nc.resample(time='1M').median('time')
rgb(ds_resampled_median, bands=['red', 'green', 'blue'], col='time')

In [ ]:
ds_max = ds_nc.NDVI.max('time')

# View the resulting composite
ds_max.plot(vmin=-1, vmax=1, cmap='RdYlGn', figsize=(10,7));

### Example 1: Phenology

In [ ]:
gdf = gpd.read_file('example_data/example2.shp')
plot.map_polygon(gdf)

In [ ]:
longitude_extents = (gdf.total_bounds[0] , gdf.total_bounds[2] )
latitude_extents  = (gdf.total_bounds[1] , gdf.total_bounds[3] )

# select time period
time_extents=('2020-01-01', '2020-12-29')

# select platform
platform = "SENTINEL_2"

# select product
product = "s2_l2a_namibia"

dataset = dc.load(latitude = latitude_extents,
                             longitude = longitude_extents,
                             time = time_extents,
                             platform =platform,
                             skip_broken_datasets=True,
                             measurements = ['red','green','blue', 'nir', 'scl'],
                             product = product,
                             group_by = "solar_day",
                             progress_cbk=with_ui_cbk()) 

In [ ]:
mean = dataset.mean(dim=['x','y'])
dataset = dataset.sel(time = pd.to_datetime(mean.time.where(mean['red'] !=0, drop=True).values.tolist()))
dataset

In [ ]:
timesteps = [0, 10, 20, 30, 40, 50]
rgb(dataset,bands=['red', 'green', 'blue'], index=timesteps)

In [ ]:
def mask(dataset): #input options: low - filter out cloud of high probability only (default); high - filter out cloud of high and medium probability
    cloud_free = dataset.where((dataset.scl != 9) & (dataset.scl != 3) & (dataset.scl != 8) &  (dataset.scl != 10))
    #new_ds = cloud_free.where((dataset.blue < 12000) & (dataset.red < 12000) & (dataset.green < 12000)) #pixels of good quality
    return cloud_free

ds_nc = mask(dataset)

In [ ]:
spectralindices(ds_nc ,indices=["NDVI", "EVI","SAVI"], norm=True, drop=False)

In [ ]:
ds_nc.NDVI.isel(time=[10,20,30,40,50]).plot(col="time")

In [ ]:
ds_nc.NDVI.mean(['x', 'y']).plot.line('b-^', figsize=(11,4))

plt.title('Zonal mean of vegetation timeseries');

In [ ]:
ds_nc.red.isel(time=[60]).plot()

In [ ]:
from dea_tools.spatial import xr_rasterize

mask = xr_rasterize(gdf, ds_nc.NDVI)
data_masked = ds_nc.where(mask)
data_masked

In [ ]:
rgb(data_masked, bands=['red', 'green', 'blue'], index = [8,9,10,11])

In [ ]:
mask = xr_rasterize(gdf.iloc[[6]], ds_nc.NDVI)
field = ds_nc.where(mask)
field

In [ ]:
rgb(field, bands=['red', 'green', 'blue'], index = [8])

In [ ]:
field.NDVI.mean(['x', 'y']).plot.line('b-^', figsize=(11,4))

plt.title('Zonal mean of vegetation timeseries');

In [ ]:
rgb(field, bands=['red', 'green', 'blue'], index = [60])

In [ ]:
df = field['NDVI'].to_dataframe("ndvi")
df = df.reset_index()
df["Month"] = df.time.dt.month
df["Year"] = df.time.dt.year
#Preview changes
df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df1= df.groupby('Month')['ndvi'].agg([pd.np.min, pd.np.max, pd.np.mean])
df1=df1.reset_index()
sns.set(style="darkgrid")
plt.figure(figsize=(10, 5))

median_vals = df.groupby('Month')['ndvi'].median()
norm = plt.Normalize(median_vals.min(), median_vals.max())
colors = plt.cm.RdYlGn(median_vals)

sns_plot = sns.boxplot(x='Month', y='ndvi', data=df  ,palette=colors)
s=sns_plot.get_figure()

### Example 2: Water

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:46057")
client

In [ ]:
gdf = gpd.read_file('example_data/water.shp')
plot.map_polygon(gdf)

In [ ]:
longitude_extents = (gdf.total_bounds[0] , gdf.total_bounds[2] )
latitude_extents  = (gdf.total_bounds[1] , gdf.total_bounds[3] )

# select time period
time_extents=('2020-01-01', '2020-12-29')

# select platform
platform = "SENTINEL_2"

# select product
product = "s2_l2a_namibia"

dataset = dc.load(latitude = latitude_extents,
                             longitude = longitude_extents,
                             time = time_extents,
                             platform =platform,
                             skip_broken_datasets=True,
                             measurements = ['red','green','blue', 'nir', 'scl','swir1','red_edge1'],
                             product = product,
                            dask_chunks={"time": 1, 'x': 1000, 'y': 1000},
                             group_by = "solar_day",
                             progress_cbk=with_ui_cbk()) 

In [ ]:
dataset_clean = dataset.sel(time = pd.to_datetime(dataset.mean(dim=['x','y']).time.where(dataset.mean(dim=['x','y'])['red'] !=0, drop=True).values.tolist()))
dataset_clean

In [ ]:
dataset_clean.red.values

### Water body detection via NDWI and NDVI

In [ ]:
data_step1 = dataset.isel(time=2)
data_step1

In [ ]:
rgb(data_step1.compute(), bands=['red', 'green', 'blue'])

In [ ]:
data_step1 = data_step1.assign(MNDWI = (data_step1["green"] - data_step1["swir1"])/(data_step1["green"] + data_step1["swir1"]))

data_step1.MNDWI.plot()

In [ ]:
data_step1 = data_step1.assign(NDVI = (data_step1["nir"] - data_step1["red"])/(data_step1["nir"] + data_step1["red"]))

data_step1.NDVI.plot()

In [ ]:
data_step1 = data_step1.assign(water = xr.where((data_step1["NDVI"] < 0) & (data_step1["MNDWI"] > 0), 1.0, 0.0))
data_step1 = data_step1.assign(nonwater = xr.where((data_step1["NDVI"] > 0) & (data_step1["MNDWI"] < 0), 1.0, 0.0))
data_step1.water.plot()

In [ ]:
import numpy as np
area = np.count_nonzero(data_step1.water.values) * 100 / 1000000 #convert unit to square kilometers

print("Water pixels for the first month: ",area," km²")

In [ ]:
water_body = data_step1.water > 0

In [ ]:
from shapely.geometry import shape
import rasterio
dtype='float32'

vectors = rasterio.features.shapes(source=data_step1.water.values.astype(dtype),transform=data_step1.geobox.transform, mask=data_step1.water.values==1)

crs =data_step1.crs
vectors = list(vectors) 
polygons = [polygon for polygon, value in vectors]
values = [value for polygon, value in vectors]
polygons = [shape(polygon) for polygon in polygons]
gdf = gpd.GeoDataFrame(data={'water': values},
                           geometry=polygons,
                           crs=str(crs))

In [ ]:
gdf.plot()

### Let's scale it up

In [ ]:
data_month = dataset_clean.sel(time='2020-6') 
rgb(data_month.compute(), bands=['swir1', 'nir', 'green'], col="time")

In [ ]:
#a function that returns cloud and defect pixels mask
def mask(dataset): #input options: low - filter out cloud of high probability only (default); high - filter out cloud of high and medium probability
    cloud_free = dataset.where((dataset.scl != 9) & (dataset.scl != 3))
    new_ds = cloud_free.where((dataset.blue < 12000) & (dataset.red < 12000) & (dataset.green < 12000)) #pixels of good quality
    return new_ds

In [ ]:
ds_nc = mask(dataset_clean)

In [ ]:
masked_data_month = mask(ds_nc ).sel(time='2020-02')
rgb(masked_data_month.compute(), bands=['red', 'green', 'blue'], col="time") 

In [ ]:
data_monthly = ds_nc.resample(time='1M').mean()
data_monthly
month = data_monthly.time.dt.month
data_monthly.coords["time"]

In [ ]:
rgb(data_monthly.compute(), bands=['swir1', 'nir', 'green'], col="time") 

In [ ]:
data_monthly = data_monthly.assign(MNDWI = (data_monthly["green"] - data_monthly["swir1"])/(data_monthly["green"] + data_monthly["swir1"]))
data_monthly = data_monthly.assign(NDVI = (data_monthly["nir"] - data_monthly["red"])/(data_monthly["nir"] + data_monthly["red"]))

In [ ]:
data_monthly = data_monthly.assign(water = xr.where((data_monthly["NDVI"] < 0) & (data_monthly["MNDWI"] > 0), 1.0, 0.0))
data_monthly = data_monthly.assign(water_null = xr.where((data_monthly["NDVI"] < 0) & (data_monthly["MNDWI"] > 0), 1.0, None))

#data_monthly = data_monthly.assign(water = xr.where((data_monthly["MNDWI"] > 0), 1.0, 0.0))
#data_monthly = data_monthly.assign(water_null = xr.where((data_monthly["MNDWI"] > 0), 1.0, None))

In [ ]:
data_monthly.water.plot(x="x", y="y", col="time", col_wrap=4, cmap='Blues')

In [ ]:
ts = data_monthly.time.to_series()

#save the water area
water_area = data_monthly.water_null.groupby("time").count({"x","y"}).compute().values * 100 / 1000000

#pandas series of water occurence
#data_monthly.water.to_series()

month = pd.Series(ts).values #get the series values
water = pd.Series(water_area).values #get the series values

frame = { 'month': month, 'water_area_km2': water }   #set up a data frame
df = pd.DataFrame(frame) 
df.index = pd.to_datetime(df["month"],format='%Y%m%d') #set up the date time index
df = df.drop(columns=["month"]) #drop extra column
df

In [ ]:
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset

fig, axes = plt.subplots(1, 1, figsize=(12, 4))
df.water_area_km2.plot(linestyle=":", marker="s", color="k")
axes.set_xlabel("Date")
axes.set_ylabel("Waterbody area (km$^2$)")
plt.show()

### Water Quality

In [ ]:
gdf = gpd.read_file('example_data/swakoppoort.shp')
plot.map_polygon(gdf)

In [ ]:
longitude_extents = (gdf.total_bounds[0] , gdf.total_bounds[2] )
latitude_extents  = (gdf.total_bounds[1] , gdf.total_bounds[3] )

# select time period
time_extents=('2020-01-01', '2020-12-29')

# select platform
platform = "SENTINEL_2"

# select product
product = "s2_l2a_namibia"

dataset = dc.load(latitude = latitude_extents,
                             longitude = longitude_extents,
                             time = time_extents,
                             platform =platform,
                             skip_broken_datasets=True,
                             measurements = ['red','green','blue', 'nir', 'scl','swir1','red_edge1'],
                             product = product,
                            dask_chunks={"time": 1, 'x': 1000, 'y': 1000},
                             group_by = "solar_day",
                             progress_cbk=with_ui_cbk()) 

In [ ]:
dataset_clean = dataset.sel(time = pd.to_datetime(dataset.mean(dim=['x','y']).time.where(dataset.mean(dim=['x','y'])['red'] !=0, drop=True).values.tolist()))
dataset_clean

In [ ]:
rgb(dataset_clean, bands=['swir1', 'nir', 'blue'], index = 61) 

In [ ]:
dataset_clean['NDCI'] = ((dataset_clean.red_edge1 - dataset_clean.red) / (dataset_clean.red_edge1 + dataset_clean.red))
dataset_clean['MNDWI'] =(dataset_clean["green"] - dataset_clean["swir1"])/(dataset_clean["green"] + dataset_clean["swir1"])

In [ ]:
dataset_clean['NDCI'].isel(time=61).plot()

In [ ]:
#a function that returns cloud and defect pixels mask
def mask(dataset): #input options: low - filter out cloud of high probability only (default); high - filter out cloud of high and medium probability
    cloud_free = dataset.where((dataset.scl != 9) & (dataset.scl != 3))
    new_ds = cloud_free.where((dataset.blue < 12000) & (dataset.red < 12000) & (dataset.green < 12000)) #pixels of good quality
    return new_ds

In [ ]:
ds_nc = mask(dataset_clean)

In [ ]:
# Constants for calculating waterbody area
pixel_length = 10  # in metres
m_per_km = 1000  # conversion from metres to kilometres

area_per_pixel = pixel_length**2 / m_per_km**2

In [ ]:
# Filter the data to contain only pixels classified as water
ds_waterarea = ds_nc.where(ds_nc.MNDWI > 0.0)

# Calculate the total water area (in km^2)
waterarea = (
    ds_waterarea.MNDWI.count(dim=["x", "y"])
    .rolling(time=3, center=True, min_periods=1)
    .median(skipna=True)
    * area_per_pixel
).persist()

# Plot the resulting water area through time
fig, axes = plt.subplots(1, 1, figsize=(12, 4))
waterarea.plot(linestyle=":", marker="s", color="k")
axes.set_xlabel("Date")
axes.set_ylabel("Waterbody area (km$^2$)")
plt.show()

In [ ]:
# Calculate the average NDCI
average_ndci = ds_waterarea.NDCI.mean(dim=["x", "y"], skipna=True).persist()

# Plot average NDCI through time
fig, axes = plt.subplots(1, 1, figsize=(12, 4))
average_ndci.plot(linestyle=":", marker="s", color="k")
axes.set_xlabel("Date")
axes.set_ylabel("Average NDCI")
plt.show()

In [ ]:
import matplotlib

# Set up the figure
fig, axes = plt.subplots(1, 1, figsize=(12, 4))

# Set the colour map to use and the normalisation. NDCI is plotted on a scale
# of -0.1 to 0.5 so the colour map is normalised to these values
min_ndci_scale = -0.1
max_ndci_scale = 0.5
cmap = plt.get_cmap("cividis")
normal = plt.Normalize(vmin=min_ndci_scale, vmax=max_ndci_scale)

# Store the dates from the data set as numbers for ease of plotting
dates = matplotlib.dates.date2num(ds_waterarea.time.values)

# Add the basic plot to the figure
# This is just a line showing the area of the waterbody over time
axes.plot_date(x=dates, y=waterarea, color="black", linestyle="-", marker="")

# Fill in the plot by looping over the possible threshold values and filling
# the areas that are greater than the threshold
color_vals = np.linspace(0, 1, 100)
threshold_vals = np.linspace(min_ndci_scale, max_ndci_scale, 100)
for ii, thresh in enumerate(threshold_vals):
    im = axes.fill_between(dates,
                           0,
                           waterarea,
                           where=(average_ndci >= thresh),
                           norm=normal,
                           facecolor=cmap(color_vals[ii]),
                           alpha=1)

# Add the colour bar to the plot
cax, _ = matplotlib.colorbar.make_axes(axes)
cb2 = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap, norm=normal)
cb2.set_label("NDCI")

# Add titles and labels to the plot
axes.set_xlabel("Date")
axes.set_ylabel("Waterbody area (km$^2$)")
plt.show()

In [ ]:
def closest_date(list_of_dates, desired_date):
    return min(list_of_dates,
               key=lambda x: abs(x - np.datetime64(desired_date)))

def date_index(list_of_dates, known_date):
    return (np.where(list_of_dates == known_date)[0][0])

In [ ]:
# Set the dates to view
date_1 = "2020-02-07"
date_2 = "2020-12-20"

# Compute the closest date available from the data set
closest_date_1 = closest_date(ds_nc.time.values, date_1)
closest_date_2 = closest_date(ds_nc.time.values, date_2)

# Make an xarray containing the closest dates, which is used to select
# the dates from the data set
time_xr = xr.DataArray([closest_date_1, closest_date_2], dims=["time"])

# Plot the NDCI values for pixels classified as water for the two dates.
ds_waterarea.NDCI.sel(time=time_xr).plot.imshow("x",
                                                   "y",
                                                   col="time",
                                                   cmap=cmap,
                                                   vmin=min_ndci_scale,
                                                   vmax=1,
                                                   col_wrap=2,
                                                   robust=True,
                                                   figsize=(12, 6))
plt.show()